In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from statsmodels.formula.api import ols
import statsmodels.api as sm
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('data/traindata.csv')

In [3]:
#Changing Uhrsmain to Ahrsmain and utohrs to Atothrs

#Does not work

# Generate transformed variables
df['ln_hrlyearn'] = np.log(df['hrlyearn'])
df['age_12_sq'] = df['age_12'] ** 2
df['tenure_sq'] = df['tenure'] ** 2
df['educ_tenure'] = df['educ'] * df['tenure']
df['naics_noc'] = df['naics_21'] * df['noc_10']

# ============ Linear Regression ============
# Create the formula string for regression
formula = """ln_hrlyearn ~ age_12 + sex + marstat + educ + noc_10 + 
             ahrsmain + atothrs + hrsaway + unpaidot + xtrahrs + 
             tenure + estsize + firmsize + efamtype + 
             C(prov) + C(naics_21) + 
             age_12_sq + tenure_sq + educ_tenure + naics_noc"""

# Fit the regression model with robust standard errors
linear_model = ols(formula, data=df).fit()
robust_results = linear_model.get_robustcov_results(cov_type='HC1')

# Print linear regression summary
print("Linear Regression Results:")
print(robust_results.summary())

# ============ Gradient Boosting Model ============
# Prepare data for Gradient Boosting
X = pd.concat([
    df[['age_12', 'sex', 'marstat', 'educ', 'noc_10', 
        'ahrsmain', 'atothrs', 'hrsaway', 'unpaidot', 'xtrahrs',
        'tenure', 'estsize', 'firmsize', 'efamtype',
        'age_12_sq', 'tenure_sq', 'educ_tenure', 'naics_noc']],
    pd.get_dummies(df['prov'], prefix='prov'),
    pd.get_dummies(df['naics_21'], prefix='naics')
], axis=1)

y = df['ln_hrlyearn']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Initialize and train Gradient Boosting model
gb_model = GradientBoostingRegressor(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=6,
    min_samples_split=5,
    random_state=42
)

# Fit the model
gb_model.fit(X_train, y_train)

# Make predictions
y_pred = gb_model.predict(X_test)

# Calculate metrics
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("\nGradient Boosting Model Results:")
print(f"Root Mean Squared Error: {rmse:.4f}")
print(f"R-squared Score: {r2:.4f}")



# Compare predictions
comparison_df = pd.DataFrame({
    'Actual': y_test,
    'Predicted': y_pred
})

print("\nPrediction Sample:")
print(comparison_df.head())

Linear Regression Results:
                            OLS Regression Results                            
Dep. Variable:            ln_hrlyearn   R-squared:                       0.457
Model:                            OLS   Adj. R-squared:                  0.456
Method:                 Least Squares   F-statistic:                     902.1
Date:                Tue, 14 Jan 2025   Prob (F-statistic):               0.00
Time:                        10:16:12   Log-Likelihood:                -12804.
No. Observations:               42092   AIC:                         2.570e+04
Df Residuals:                   42044   BIC:                         2.612e+04
Df Model:                          47                                         
Covariance Type:                  HC1                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept  